In [1]:
import os 
os.chdir('..')

In [294]:
import pandas as pd
import numpy as np
import scipy.stats as st
from tqdm import tqdm
import itertools 
from matplotlib import pyplot as plt
import pickle as pkl


from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import median_absolute_error


In [38]:
from src.elegant_fuzzy_genetic_algorithms.helpers.generalized_param_inferrer import GeneralizedParamInferencer
from src.elegant_fuzzy_genetic_algorithms.helpers.all_params_wrapper import AllEFGAParamsParallelWrapper
from src.elegant_fuzzy_genetic_algorithms.param_diff_simulation import simulation_param_diff
from src.common.fitness import (griewank, schwefel, ackley, rastrigin)
from src.common.approximation_helpers import generate_search_space, init_param_index
from src.common.scales import scales

In [12]:
gpi = AllEFGAParamsParallelWrapper(3, 3)

In [16]:
priorities = []

for fn in [griewank, schwefel, rastrigin, ackley]:
    fn_name = fn.__name__
    priorities_ = simulation_param_diff(N=100, epochs=200, fitness_fn=fn, population_scale=scales[fn_name][0], 
                      mutation_scale=scales[fn_name][1], seed=1)
    priorities.append(priorities_)

priorities_df = list(itertools.chain(*priorities))
priorities_df = pd.DataFrame(priorities_df)

100%|██████████| 200/200 [00:03<00:00, 51.69it/s]


In [17]:
priorities_df

,xRate,mRate,subPopSize,best_fitness,avg_fitness,avg_fit_change
0,0.668841,0.022403,0.185050,0.100325,0.265879,0.037544
1,0.663993,0.021999,0.185809,0.100325,0.249000,0.015726
2,0.665648,0.022137,0.185554,0.100325,0.254578,0.006794
3,0.665151,0.022096,0.185631,0.100325,0.252894,0.003249
4,0.665498,0.022125,0.185577,0.100325,0.254069,0.002857
...,...,...,...,...,...,...
795,0.857145,0.038095,0.216938,0.929401,0.964470,0.000158
796,0.855993,0.037999,0.216926,0.929401,0.963351,0.000435
797,0.857113,0.038093,0.216938,0.929401,0.964439,0.000327
798,0.857614,0.038135,0.216944,0.929401,0.964924,0.000502


In [18]:
param_chg = np.vstack([np.diff(pd.DataFrame(priorities[i])[['xRate', 'mRate', 'subPopSize']].to_numpy(), axis=0) for i in range(len(priorities))])

In [21]:
keys = list(priorities[0][0].keys())[:3]

for i in range(len(keys)):
    param_stats = np.abs(param_chg[:, i])
    conf_int = st.t.interval(alpha=0.95, df=len(param_stats)-1, loc=np.mean(param_stats), scale=st.sem(param_stats)) 
    print(f'{keys[i]=}, {conf_int=}')

keys[i]='xRate', conf_int=(0.0010480190433324948, 0.0013090614472372334)
keys[i]='mRate', conf_int=(8.733492027770745e-05, 0.00010908845393643577)
keys[i]='subPopSize', conf_int=(7.0857761053587e-05, 9.843435710761442e-05)


The goal in this case is to obtain max error confidence interval higher range lower than lower interval of error

In [74]:
priorities_df.best_fitness.min(), priorities_df.best_fitness.max()

(0.10032480226915341, 0.9294013403865408)

In [75]:
priorities_df.avg_fitness.min(), priorities_df.avg_fitness.max()

(0.015014670631496705, 0.972623828244773)

In [77]:
priorities_df.avg_fit_change.min(), priorities_df.avg_fit_change.max()

(0.0, 0.03754400608188846)

In [150]:
search_space = generate_search_space((20, 20, 20), [(priorities_df.best_fitness.min(), priorities_df.best_fitness.max(),), 
                                           (priorities_df.avg_fitness.min(), priorities_df.avg_fitness.max()), 
                                           (priorities_df.avg_fit_change.min(), priorities_df.avg_fit_change.max())])


In [151]:
y = [gpi.infer(*search_space[i]) for i in tqdm(range(search_space.shape[0]))]

100%|██████████| 8000/8000 [01:48<00:00, 73.43it/s] 


In [128]:
y_ml = pd.DataFrame(y)['xRate']

In [134]:
! pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.4 MB/s eta 0:00:0000:0100:01


In [129]:
from sklearn.neural_network import MLPRegressor

In [136]:
from xgboost import XGBRegressor

In [152]:
mlpr = XGBRegressor(n_estimators=10000).fit(search_space, y_ml)

In [115]:
N = 2000
np.random.seed(1)
test_set = np.vstack([np.random.uniform(priorities_df.best_fitness.min(), priorities_df.best_fitness.max(), N), 
                      np.random.uniform(priorities_df.avg_fitness.min(), priorities_df.avg_fitness.max(), N), 
                    np.random.uniform(priorities_df.avg_fit_change.min(), priorities_df.avg_fit_change.max(), N), 

                      ]).T
test_set

array([[0.44606796, 0.56753394, 0.01347252],
       [0.69752894, 0.85329494, 0.02454445],
       [0.10041963, 0.59778251, 0.02127211],
       ...,
       [0.78740777, 0.08534165, 0.00098332],
       [0.12602448, 0.57017618, 0.03569789],
       [0.885235  , 0.71743728, 0.02139699]])

In [116]:
y_actual = [gpi.infer(*test_set[i]) for i in tqdm(range(test_set.shape[0]))]







































































































































































































































































































































































































































































































































































100%|██████████| 2000/2000 [00:30<00:00, 65.20it/s]


In [156]:
mlpr.predict(test_set)[:, 0]

array([0.7325925 , 0.74321914, 0.679307  , ..., 0.6850797 , 0.7435712 ,
       0.72447383], dtype=float32)

In [117]:
param_index = init_param_index(search_space)

In [118]:
np.array(y)

array([{'xRate': 0.5197097290295171, 'mRate': 0.009975810752459647, 'subPopSize': 0.19903609760090574},
       {'xRate': 0.5197097290295171, 'mRate': 0.009975810752459647, 'subPopSize': 0.19903609760090574},
       {'xRate': 0.5197097290295171, 'mRate': 0.009975810752459647, 'subPopSize': 0.19903609760090574},
       ...,
       {'xRate': 0.8662411916514636, 'mRate': 0.03885343263762199, 'subPopSize': 0.21714114285714275},
       {'xRate': 0.8662411916514636, 'mRate': 0.03885343263762199, 'subPopSize': 0.21714114285714275},
       {'xRate': 0.8662411916514636, 'mRate': 0.03885343263762199, 'subPopSize': 0.21714114285714275}],
      dtype=object)

In [119]:
D, I = param_index.search(test_set, k=1)


In [120]:
y_pred = pd.DataFrame(list(itertools.chain(*np.array(y)[I].tolist())))

In [121]:
diff_df = y_pred - pd.DataFrame.from_records(y_actual)

In [122]:
for col in diff_df.columns:
    diff_df[col] = np.abs(diff_df[col])
    diff_arr = diff_df[col]
    conf_int = st.t.interval(alpha=0.95, df=len(diff_arr)-1, loc=np.mean(diff_arr), scale=st.sem(diff_arr)) 
    print(col, conf_int)

    

xRate (0.004724630914909042, 0.005170528347057861)
mRate (0.00039371924290909253, 0.00043087736225482756)
subPopSize (0.0013664122238693452, 0.001503761347596933)


In [293]:
for col in ['xRate', 'mRate', 'subPopSize']:
    X_train, X_test, y_train, y_test = train_test_split(priorities_df[['best_fitness', 'avg_fitness', 'avg_fit_change']], 
                                                        priorities_df[col])
    mlpr = XGBRegressor(tree_method="hist",).fit(X_train, y_train)
    y_pred = mlpr.predict(X_test)
    print(f'Rate per 1000: {median_absolute_error(y_pred, y_test) * 1000}')
    print(f'Bias: {median_absolute_error(y_pred, y_test)}')

Rate per 1000: 0.15865628636246853
Bias: 0.00015865628636246853
Rate per 1000: 0.05608443704228569
Bias: 5.608443704228569e-05
Rate per 1000: 0.0603986052967187
Bias: 6.03986052967187e-05


Provided that most errors are centered around 0 (there's no systematic overstating or understating of some metric), as well as rates being less than <.15 per 1000, the approximation would be considered correct. Let's train a model and then use it for the future inference. 

In [295]:
for col in ['xRate', 'mRate', 'subPopSize']:
    mlpr = XGBRegressor(tree_method="hist",).fit(priorities_df[['best_fitness', 'avg_fitness', 'avg_fit_change']], 
                                                        priorities_df[col])
    
    with open('./indices/xgb_{col}.pkl', 'wb') as f:
        pkl.dump(mlpr, f)
    